## Mixing Audio - CHiME
This file was used to create a new dataset consisting of voice, background noise and mixture, based on the clean voice and background noise provided in the CHiME 3 data set.

In [ ]:
# Imports

# This alows us to import directories which are stored at the level above this file
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

import numpy as np
import soundfile as sf
import librosa
import IPython.display as ipd
from sklearn.preprocessing import minmax_scale
from glob import glob
import re
import matplotlib.pyplot as plt
%matplotlib inline
from shutil import copy
import audio_functions as af
import dataset

In [ ]:
def mix_audio(voice_file, background_file):
    """
    Takes the location of two audio files (a voice file and a background noise file), 
    loads them, and mixes them with a 60:40 ratio. The first file must be shorter 
    than the second.

    Args:
    voice_file: String - The location of the first file.
    background_file: String - The location of the second file.

    Output:
    norm_mix: 1D array - Normalised 60:40 mixture of voice and background_segment.
    voice_sr: Int - The sample rate of the voice file (which the returned background noise also matches).
    background_segment: 1D array - A background noise segment with len = len(voice).
    voice: 1D array - the voice audio.
    """    
    # Load the files
    voice, voice_sr = sf.read(voice_file)
    background, background_sr = sf.read(background_file)
    
    # Make sure the sample rates match
    if voice_sr != background_sr:
        librosa.resample(background, voice_sr)

    # Select a segment of the background track to use
    start_point = np.random.randint(len(background) - len(voice))
    background_segment = background[start_point:start_point+len(voice)]

    # Normalise both sources and mix
    norm_voice = minmax_scale(voice, (-1,1))
    norm_bgs = minmax_scale(background_segment,(-1,1))
    norm_mix = (0.6*norm_voice) + (0.4*norm_bgs)
    
    return norm_mix, voice_sr, background_segment, voice

In [ ]:
# Copy the file structure

inputpath = '/data/Speech_Data/CHiME3/data/audio/16kHz/isolated/'
outputpath = '/home/enterprise.internal.city.ac.uk/acvn728/NewCHiME/'


for dirpath, dirnames, filenames in os.walk(inputpath):
    structure = os.path.join(outputpath, dirpath[len(inputpath):])
    if not os.path.isdir(structure):
        os.mkdir(structure)
    else:
        print("Folder already exits!")
for folder in os.listdir(outputpath):
    if '_real' in folder:
        os.rename(os.path.join(outputpath, folder),
                  os.path.join(outputpath, folder[:-4] + 'bg'))

### To Recreate the Development and Evaluation Sets:

In [ ]:
source = '/data/Speech_Data/CHiME3/data/audio/16kHz/isolated'
destination = '/home/enterprise.internal.city.ac.uk/acvn728/NewCHiME'
for directory in ['dt05_bth', 'et05_bth', 'tr05_org']:
    file_list = os.listdir(os.path.join(source,directory))
    for file in file_list:
        copy(os.path.join(source,directory,file), os.path.join(destination,directory))

In [ ]:
count = 0
background_file_list = glob('/data/Speech_Data/CHiME3/data/audio/16kHz/backgrounds/*.wav')
count = 0
for directory in ['/home/enterprise.internal.city.ac.uk/acvn728/NewCHiME/dt05_bth',
                  '/home/enterprise.internal.city.ac.uk/acvn728/NewCHiME/et05_bth']:
    # Get a list of the files
    folder_name = directory[-8:]
    file_list = os.listdir(directory)
    # Leave out CH0 as they do not exist in the backgruond noise recordings
    file_list = [f for f in file_list if re.search('CH0', f) is None]
    
    # Loop over files
    for file in file_list:
        for env in ['bus', 'caf', 'ped', 'str']:
            # Work out the names and destinations for the new background and mixture files
            new_file_name = file.split('BTH')[0] + env.upper() + file.split('BTH')[1]
            new_mix_folder_name = folder_name[:-3] + env + '_simu'
            new_mix_file_path = os.path.join(directory[:-8], new_mix_folder_name, new_file_name)
            new_bg_folder_name = folder_name[:-3] + env + '_bg'
            new_bg_file_path = os.path.join(directory[:-8], new_bg_folder_name, new_file_name) 
            # Check if file already exists
            if os.path.isfile(new_mix_file_path):
                print('File already exists!')
                count += 1
                continue
            # Select a background file
            suitable_files = [f for f in background_file_list if f[-7:] == new_file_name[-7:]] 
            background_file = suitable_files[np.random.randint(len(suitable_files))]
            # Mix them
            mixture, sample_rate, background_noise, _ = mix_audio(os.path.join(directory, file), background_file)
            # Save to file
            sf.write(new_mix_file_path, mixture, sample_rate)
            sf.write(new_bg_file_path, background_noise, sample_rate)
            count += 1
            if count % 1000 == 0:
                print('{c} files processed.'.format(c=count))

### To Recreate the Training Set
(which has a slightly different naming convention)

In [ ]:
background_file_list = glob('/data/Speech_Data/CHiME3/data/audio/16kHz/backgrounds/*.wav')
original_voice_directory = '/data/Speech_Data/CHiME3/data/audio/16kHz/isolated/tr05_org'
directory = '/home/enterprise.internal.city.ac.uk/acvn728/NewCHiME/tr05_org'
folder_name = directory[-8:]
count = 0

for env in ['bus', 'caf', 'ped', 'str']:
    orig_sim_directory = original_voice_directory[:-3] + env + '_simu'
    file_list = os.listdir(orig_sim_directory)
    file_list = set([f[:-11] for f in file_list])

    # Loop over files
    for file in file_list:
        for i in range(1,7):
            # Work out the names and destinations for the new background and mixture files
            new_file_name = file + env.upper() + '.CH{num}.wav'.format(num=str(i))
            new_mix_folder_name = folder_name[:-3] + env + '_simu'
            new_mix_file_path = os.path.join(directory[:-8], new_mix_folder_name, new_file_name)
            new_bg_folder_name = folder_name[:-3] + env + '_bg'
            new_bg_file_path = os.path.join(directory[:-8], new_bg_folder_name, new_file_name)
            # Check if file already exists
            if os.path.isfile(new_mix_file_path):
                print('File already exists!')
                count += 1
                continue
            # Select a background file
            suitable_files = [f for f in background_file_list if f[-11:] == new_file_name[-11:]] 
            background_file = suitable_files[np.random.randint(len(suitable_files))]
            # Mix them
            mixture, sample_rate, background_noise, _ = mix_audio(os.path.join(directory, file + 'ORG.wav'), background_file)
            # Save to file
            sf.write(new_mix_file_path, mixture, sample_rate)
            sf.write(new_bg_file_path, background_noise, sample_rate)
            count += 1
            if count % 1000 == 0:
                print('{c} files processed.'.format(c=count))